In [1]:
import pandas as pd

In [2]:
from scipy import spatial
import scipy as sp

In [3]:
import geopy
import pyproj

In [4]:
import math

def cartesian(latitude, longitude, elevation = 0):
    # Convert to radians
    latitude = latitude * (math.pi / 180)
    longitude = longitude * (math.pi / 180)

    R = 6371 # 6378137.0 + elevation  # relative to centre of the earth
    X = R * math.cos(latitude) * math.cos(longitude)
    Y = R * math.cos(latitude) * math.sin(longitude)
    Z = R * math.sin(latitude)
    return (X, Y, Z)

In [87]:
apteky = pd.read_csv("pharmacy_all_contracted_legal_entities.csv")
hospitals = pd.read_csv("pmd_all_contracted_legal_entities.csv")
declarations = pd.read_csv('all_declarations_pmd.csv', dtype={'division_settlement_koatuu': str})

/home/ptrbdr/files/medical_reform/env/lib/python3.5/site-packages/IPython/core/interactiveshell.py:3049: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [88]:
declarations['division_decl_sum'] = declarations.groupby('division_id')['count_declarations'].transform('sum')

In [89]:
d_short = declarations[['legal_entity_id','division_id', 'division_settlement_koatuu', 'division_decl_sum']]

In [106]:
t = pd.merge(hospitals, d_short, on=['division_id'], how='left')

# Півень відповів, що потрібно відкидати тих хто не мерджаться:
# Вірно, не всі підрозділи з деклараціями. 
# Ті де нема декларацій краще не додавати на карти, оскільки ПМД там не надається посуті

combined_hospitas = t[~t.division_decl_sum.isnull()].drop_duplicates()

In [107]:
coords = combined_hospitas[['nszu_geocoding_google_api_lat', 'nszu_geocoding_google_api_lng']].apply(tuple, axis=1)

In [111]:
hospitals_short = combined_hospitas[['division_id', 'nszu_geocoding_google_api_lat', 'division_residence_addresses',
                             'division_name', 'division_settlement_koatuu', 'division_decl_sum',
                             'nszu_geocoding_google_api_lng']]
hospitals_short['coords'] = coords
hospitals_short['coords'] = [cartesian(*d) for d in hospitals_short.coords.values]

/home/ptrbdr/files/medical_reform/env/lib/python3.5/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.
/home/ptrbdr/files/medical_reform/env/lib/python3.5/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """


In [112]:
hospitals_tree = spatial.cKDTree([cartesian(*d) for d in hospitals[['nszu_geocoding_google_api_lat', 'nszu_geocoding_google_api_lng']].values])
apteky_tree = spatial.cKDTree([cartesian(*d) for d in apteky[['nszu_geocoding_google_api_lat', 'nszu_geocoding_google_api_lng']].values])

In [113]:
import ipdb 
result = []
for i, d in enumerate(hospitals_short['coords']):
    print(i)
#     ipdb.set_trace()
    l = apteky_tree.query_ball_point(d, 5)
    result.append(apteky.loc[l]['division_id'].values.tolist())
    
    
    
    

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

1948
1949
1950
1951
1952
1953
1954
1955
1956
1957
1958
1959
1960
1961
1962
1963
1964
1965
1966
1967
1968
1969
1970
1971
1972
1973
1974
1975
1976
1977
1978
1979
1980
1981
1982
1983
1984
1985
1986
1987
1988
1989
1990
1991
1992
1993
1994
1995
1996
1997
1998
1999
2000
2001
2002
2003
2004
2005
2006
2007
2008
2009
2010
2011
2012
2013
2014
2015
2016
2017
2018
2019
2020
2021
2022
2023
2024
2025
2026
2027
2028
2029
2030
2031
2032
2033
2034
2035
2036
2037
2038
2039
2040
2041
2042
2043
2044
2045
2046
2047
2048
2049
2050
2051
2052
2053
2054
2055
2056
2057
2058
2059
2060
2061
2062
2063
2064
2065
2066
2067
2068
2069
2070
2071
2072
2073
2074
2075
2076
2077
2078
2079
2080
2081
2082
2083
2084
2085
2086
2087
2088
2089
2090
2091
2092
2093
2094
2095
2096
2097
2098
2099
2100
2101
2102
2103
2104
2105
2106
2107
2108
2109
2110
2111
2112
2113
2114
2115
2116
2117
2118
2119
2120
2121
2122
2123
2124
2125
2126
2127
2128
2129
2130
2131
2132
2133
2134
2135
2136
2137
2138
2139
2140
2141
2142
2143
2144
2145
2146
2147


3604
3605
3606
3607
3608
3609
3610
3611
3612
3613
3614
3615
3616
3617
3618
3619
3620
3621
3622
3623
3624
3625
3626
3627
3628
3629
3630
3631
3632
3633
3634
3635
3636
3637
3638
3639
3640
3641
3642
3643
3644
3645
3646
3647
3648
3649
3650
3651
3652
3653
3654
3655
3656
3657
3658
3659
3660
3661
3662
3663
3664
3665
3666
3667
3668
3669
3670
3671
3672
3673
3674
3675
3676
3677
3678
3679
3680
3681
3682
3683
3684
3685
3686
3687
3688
3689
3690
3691
3692
3693
3694
3695
3696
3697
3698
3699
3700
3701
3702
3703
3704
3705
3706
3707
3708
3709
3710
3711
3712
3713
3714
3715
3716
3717
3718
3719
3720
3721
3722
3723
3724
3725
3726
3727
3728
3729
3730
3731
3732
3733
3734
3735
3736
3737
3738
3739
3740
3741
3742
3743
3744
3745
3746
3747
3748
3749
3750
3751
3752
3753
3754
3755
3756
3757
3758
3759
3760
3761
3762
3763
3764
3765
3766
3767
3768
3769
3770
3771
3772
3773
3774
3775
3776
3777
3778
3779
3780
3781
3782
3783
3784
3785
3786
3787
3788
3789
3790
3791
3792
3793
3794
3795
3796
3797
3798
3799
3800
3801
3802
3803


5243
5244
5245
5246
5247
5248
5249
5250
5251
5252
5253
5254
5255
5256
5257
5258
5259
5260
5261
5262
5263
5264
5265
5266
5267
5268
5269
5270
5271
5272
5273
5274
5275
5276
5277
5278
5279
5280
5281
5282
5283
5284
5285
5286
5287
5288
5289
5290
5291
5292
5293
5294
5295
5296
5297
5298
5299
5300
5301
5302
5303
5304
5305
5306
5307
5308
5309
5310
5311
5312
5313
5314
5315
5316
5317
5318
5319
5320
5321
5322
5323
5324
5325
5326
5327
5328
5329
5330
5331
5332
5333
5334
5335
5336
5337
5338
5339
5340
5341
5342
5343
5344
5345
5346
5347
5348
5349
5350
5351
5352
5353
5354
5355
5356
5357
5358
5359
5360
5361
5362
5363
5364
5365
5366
5367
5368
5369
5370
5371
5372
5373
5374
5375
5376
5377
5378
5379
5380
5381
5382
5383
5384
5385
5386
5387
5388
5389
5390
5391
5392
5393
5394
5395
5396
5397
5398
5399
5400
5401
5402
5403
5404
5405
5406
5407
5408
5409
5410
5411
5412
5413
5414
5415
5416
5417
5418
5419
5420
5421
5422
5423
5424
5425
5426
5427
5428
5429
5430
5431
5432
5433
5434
5435
5436
5437
5438
5439
5440
5441
5442


In [114]:
pd.Series([len(d) for d in result]).describe()

count    6220.000000
mean       18.994695
std        34.897398
min         0.000000
25%         0.000000
50%         2.000000
75%        18.000000
max       197.000000
dtype: float64

In [115]:
hospitals_short['pharmas'] = result
hospitals_short

/home/ptrbdr/files/medical_reform/env/lib/python3.5/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


,division_id,nszu_geocoding_google_api_lat,division_residence_addresses,division_name,division_settlement_koatuu,division_decl_sum,nszu_geocoding_google_api_lng,coords,pharmas
1,0014c9f4-0869-4b07-b0c0-ef38d2454980,47.636456,"ДНІПРОПЕТРОВСЬКА область, місто ПОКРОВ, вулиця...",Амбулаторія загальної практики сімейної медици...,1212100000,7360.0,34.072742,"(3555.9961718004074, 2405.1239681216666, 4707....","[b6ddcdfe-870f-49ad-ae4a-29a312d6c27f, 7dc5dce..."
41,002e2429-3c08-4d25-b9a1-a3839ed20a98,49.931315,"ЛЬВІВСЬКА область, ЯВОРІВСЬКИЙ район, село БЕР...",Амбулаторія загальної практики-сімейної медици...,4625880401,1551.0,23.573347,"(3758.810469582065, 1640.1020429509372, 4875.5...","[5a413303-aa58-4437-ac94-264074c1a9a2, 049f0cb..."
55,0034f1ce-8ce4-47c5-9c36-6b4ffc6b2208,51.511968,"ЧЕРНІГІВСЬКА область, місто ЧЕРНІГІВ, вулиця Р...","Комунальне некомерційне підприємство ""Дитяча п...",7410100000,17237.0,31.325855,"(3386.9986642370427, 2061.4214934982915, 4986....","[b0624af6-e954-421c-901e-365cad147640, 9f82dac..."
163,0034fadd-467a-428e-824d-13ddded6be5c,49.255546,"ВІННИЦЬКА область, місто ВІННИЦЯ, вулиця Стані...",Амбулаторія загальної практики- сімейної медиц...,0510100000,5783.0,28.473527,"(3655.270975981066, 1982.4640132587474, 4826.8...","[af053050-e07c-41dd-8047-004026b53fab, 6c5e4d4..."
216,0056a422-ba39-4dcf-ae23-915fa1941be1,49.831131,"ЛЬВІВСЬКА область, місто ЛЬВІВ, вулиця Генерал...",Відділення загальної практики-сімейної медицин...,4610100000,18673.0,24.009852,"(3753.988230137197, 1672.1567809041176, 4868.3...","[79177207-5099-4d03-9afc-3282f3a12151, 2297753..."
390,0081f79b-77bc-46cb-a810-75e062c2d2e2,49.217695,"ЧЕРКАСЬКА область, місто СМІЛА, вулиця Героїв ...",Амбулаторія загальної практики-сімейної медици...,7110500000,8408.0,31.915687,"(3532.3535398873328, 2200.0385194288865, 4824....","[1ee03325-24ab-423d-80cf-b5cddd97e5de, 69d1e09..."
446,00935812-efe0-44cb-8c68-fcf01db8f19f,49.237554,"ХМЕЛЬНИЦЬКА область, ЯРМОЛИНЕЦЬКИЙ район, село...",Михайлівська амбулаторія загальної практики сі...,6825883601,2244.0,27.146830,"(3701.5399677025794, 1897.9929654312168, 4825....",[]
475,009e4e73-f051-4af6-b9d7-effa7deba9e0,47.666242,"ДНІПРОПЕТРОВСЬКА область, ШИРОКІВСЬКИЙ район, ...",Карпівська амбулаторія загальної практики сіме...,1225855305,1225.0,33.119047,"(3593.485169865905, 2344.266148144803, 4709.66...","[b5295505-0374-432d-838b-7672e9187c49, 922fe7d..."
486,00a605e5-95bd-4357-b099-92ed69609838,49.818865,"ЛЬВІВСЬКА область, місто ЛЬВІВ, вулиця Виговсь...",Амбулаторія сімейної медицини № 1 центру перви...,4610100000,11083.0,23.970439,"(3756.0898681110434, 1669.9974024878118, 4867....","[79177207-5099-4d03-9afc-3282f3a12151, 79432c8..."
576,00c59864-26ef-4474-af4c-cf0761585d6f,50.691709,"СУМСЬКА область, КРАСНОПІЛЬСЬКИЙ район, село Р...",АЗПСМ с. Рясне,5922385001,1060.0,35.395038,"(3290.0442844037884, 2337.6839692272097, 4929....",[]


In [14]:
# hospitals_and_pharmacies = dict(zip(hospitals_short.division_id.values.tolist(),result,hospitals_short.division_id.values.tolist(), ))


ValueError: dictionary update sequence element #0 has length 3; 2 is required

In [121]:
hospitals_short = hospitals_short.drop(['coords', 'division_id'], axis=1)

In [126]:
hospitals_short.to_dict('records')

[{'division_decl_sum': 7360.0,
  'division_name': 'Амбулаторія загальної практики сімейної медицини №1',
  'division_residence_addresses': 'ДНІПРОПЕТРОВСЬКА область, місто ПОКРОВ, вулиця Київська, 9',
  'division_settlement_koatuu': '1212100000',
  'nszu_geocoding_google_api_lat': 47.636456,
  'nszu_geocoding_google_api_lng': 34.0727419,
  'pharmas': ['b6ddcdfe-870f-49ad-ae4a-29a312d6c27f',
   '7dc5dceb-654d-4b52-9149-22a61c07666f',
   '3643047a-01ce-4579-8860-00940f0b3076',
   '1b6a23df-6eaa-42af-82f6-0be79a5d98b4',
   'cdffa2f2-bba6-4929-8230-855a459b8b87',
   'f72d1b7b-c149-4deb-9597-a5ac5e138e6b',
   'f76511bd-1f63-4b5f-91aa-20a3602b9a6a',
   '73be4fb8-f949-4a72-8346-b561498a8bee',
   '5bf4bbc2-73b3-414c-a194-de9369d2ae8e',
   '95bfc7aa-296d-4eed-a816-e6955b7c4c8c',
   'e0d9660c-59a1-4f91-9a73-bd090080e721']},
 {'division_decl_sum': 1551.0,
  'division_name': 'Амбулаторія загальної практики-сімейної медицини с.Бердихів',
  'division_residence_addresses': 'ЛЬВІВСЬКА область, ЯВОРІВС

In [127]:
import json
with open('hospitals_and_pharmacies.json', 'w') as f:
    json.dump(hospitals_short.to_dict('records'), f)

In [58]:
pd.Series([len(d) for d in result]).describe()

count    10834.000000
mean        24.928466
std         56.627587
min          0.000000
25%          0.000000
50%          2.000000
75%         13.000000
max        359.000000
dtype: float64

In [41]:
# sparse_matrix = hospitals_tree.sparse_distance_matrix(apteky_tree, max_distance=5)